In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

In [3]:
DATE_RANGE = ('04-01-2016', '10-31-2016')

df = pd.read_csv(
    f'./data/city/Toulouse/X_hour_toulouse.csv',
    index_col=0
)
df['date'] = pd.date_range(*DATE_RANGE, freq='1h')[:len(df)]
df.set_index('date', inplace=True)

In [4]:
a = pd.Series([False, True, True, False, True], index=[0, 1, 2, 9, 10])
a[0]

False

In [5]:
def get_interpolated_indices(serie: pd.Series, tolerance: float=1e-3, output_type: str='mask'):

    mask, current_mask = [], []
    for index in range(len(serie)):
        # Si < 2 éléments d'affilé de suite
        if len(current_mask) < 2:
            current_mask.append(serie.index.values[index])
            continue
        
        # On check si la différence est la même entre trois éléments d'affilé de suite
        if np.abs((serie.values[index - 2] - serie.values[index - 1]) - (serie.values[index - 1] - serie.values[index])) < tolerance:
            current_mask.append(serie.index.values[index])
            continue
        
        # Si la différence est constante et la longueur de l'interpolation < 24 heures
        if len(current_mask) < 24 and np.abs(serie.values[index - 2] - serie.values[index - 1]) < tolerance:
            current_mask = []

        if len(current_mask) > 2:
            mask.append(current_mask[1:-1])
        current_mask = [serie.index.values[index - 1], serie.index.values[index]]

    if output_type == 'maks':
        return serie[[index_mask for row in mask for index_mask in row]].index

    if output_type == 'list':
        return mask
    
    raise ValueError('The "output_type" value must be one of the following : "mask", "list".')

In [6]:
def get_interpolated_indices(serie: pd.Series, tolerance: float=1e-9, output_type: str='mask') -> pd.Index | pd.DatetimeIndex:
    interpolated_mask = np.roll((serie.diff().abs() > tolerance) & (serie.diff().diff().abs() < tolerance), -1)

    if output_type == 'mask':
        return serie[interpolated_mask].index
    
    if output_type == 'list':
        serie_mask = pd.Series(interpolated_mask, index=serie.index.values)
        list_mask, current_list = [], []
        for index, bool_value in serie_mask.items():
            if bool_value:
                current_list.append(index)
                continue
            
            if current_list:
                list_mask.append(current_list)
                current_list = []
        return list_mask

    raise ValueError('The "output_type" value must be one of the following : "mask", "list".')

In [7]:
index_column = 0
tmp = df.loc[:].iloc[:, index_column].copy()
# tmp = pd.Series([-2, 1, 2, 3, 84, 56, 98, 5, 4, 3, 2, 92])

index = get_interpolated_indices(tmp, output_type='list')
print(len(index))
index = [u for v in index for u in v]
tmp.loc[index] = np.nan

a = tmp.to_frame()
a['origine'] = df.iloc[:, index_column].copy()
fig = px.line(a).show()

40


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

In [ ]:
def f(x):
    return str(x)

import numpy as np
np.vectorize(f)(np.array([1, 2, 3]))

array(['1', '2', '3'], dtype='<U1')

In [8]:
from data.prediction.methods.mean_predictor import predict_by_mean

train = df.loc['2016-06-01':'2016-08-01']
test = df.loc['2016-08-01':'2016-10-31']

predict_by_mean(train)



c:\Users\theo3\Documents\GitHub\TER\data\prediction\methods\mean_predictor.py:12: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



00001-poids-de-lhuile  00007-place-victor-hugo  \
days   hours                                                   
Monday 0                   0.087789                 0.213333   
       1                   0.099415                 0.213333   
       2                   0.117971                 0.223528   
       3                   0.178647                 0.275185   
       4                   0.473194                 0.424868   
...                             ...                      ...   
Sunday 19                  0.086874                 0.305355   
       20                  0.070175                 0.227673   
       21                  0.046784                 0.195556   
       22                  0.041280                 0.183613   
       23                  0.052632                 0.171880   

              00008-espace-saint-georges  00013-lois-esquile  00014-perigord  \
days   hours                                                                   
Monday 0                        0.120000            0.096296        0.294118   
       1                        0.102222            0.096296        0.331592   
       2                        0.114789            0.092061        0.349180   
       3                        0.219979            0.141033        0.322136   
       4                        0.408384            0.306812        0.507152   
...                                  ...                 ...             ...   
Sunday 19                       0.144010            0.135126        0.355302   
       20                       0.130641            0.150536        0.321297   
       21                       0.120000            0.119022        0.313725   
       22                       0.128889            0.107337        0.304363   
       23                       0.124444            0.103704        0.296370   

              00015-pont-montaudran  00017-strasbourg-sardane  \
days   hours                                                    
Monday 0                   0.455556                  0.097622   
       1                   0.444548                  0.139419   
       2                   0.410829                  0.190799   
       3                   0.309587                  0.213802   
       4                   0.219590                  0.220983   
...                             ...                       ...   
Sunday 19                  0.479089                  0.172712   
       20                  0.471174                  0.108331   
       21                  0.468226                  0.087709   
       22                  0.467071                  0.065839   
       23                  0.477778                  0.067633   

              00018-strasbourg-jean-jaures  00021-carnot-labeda  \
days   hours                                                      
Monday 0                          0.021243             0.088889   
       1                          0.044444             0.164444   
       2                          0.057431             0.197973   
       3                          0.094305             0.285536   
       4                          0.230579             0.384868   
...                                    ...                  ...   
Sunday 19                         0.082684             0.154710   
       20                         0.041110             0.115556   
       21                         0.013333             0.084444   
       22                         0.016320             0.084393   
       23                         0.007003             0.088889   

              00022-carnot-roland  ...  00271-ancely  00272-andre-vasseur  \
days   hours                       ...                                      
Monday 0                 0.075453  ...      0.366667             0.857751   
       1                 0.088202  ...      0.372222             0.807397   
       2                 0.140874  ...      0.366667             0.795231   
       3                 0.198585  ..